# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [3]:
driver = webdriver.Chrome()

In [8]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [17]:
search_box = driver.find_element_by_name("MineId")
driver.execute_script("arguments[0].scrollIntoView(true)", search_box)
search_box.clear()
search_box.send_keys("3503598")

search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [20]:
b_date = driver.find_element_by_name("BDate")
b_date.clear()
b_date.send_keys("1/1/1995")

violation_check = driver.find_elements_by_name("Submit")[3]
violation_check.click()

get_report = driver.find_elements_by_name("submit")[0]
get_report.submit()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [35]:
results = driver.find_elements_by_css_selector("tr.drsviols")
data_list = []
for result in results:
    data = {}
    info = result.find_elements_by_tag_name("td")

    data['violator'] = info[0].text
    data['contract_id'] = info[1].text
    data['citation_no'] = info[2].text
    data['case_no'] = info[3].text
    data['date_issues'] = info[4].text
    data['final_order_date'] = info[5].text
    data['section_of_act'] = info[6].text
    data['date_terminated'] = info[7].text
    data['citation'] = info[8].text
    data['s_and_s'] = info[9].text
    data['standard'] = info[10].text
    data['standard_url'] = info[10].find_element_by_tag_name("a").get_attribute("href")
    data['proposed_penalty'] = info[11].text
    data['citation_status'] = info[12].text
    data['current_penalty'] = info[13].text
    data['amount_paid'] = info[14].text
    data_list.append(data)
    
#     print("my dictionary looks like", data)
#     print("-----------------")
print(data_list)

[{'violator': 'Newberg Rock & Dirt', 'contract_id': '  ', 'citation_no': '8992368  ', 'case_no': '000440809', 'date_issues': '5/2/2017', 'final_order_date': '7/23/2017 ', 'section_of_act': '104(a)', 'date_terminated': '5/2/2017', 'citation': 'C', 's_and_s': 'N', 'standard': '56.14107(a)', 'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf', 'proposed_penalty': '116.00', 'citation_status': 'Closed', 'current_penalty': '116.00 ', 'amount_paid': '116.00 '}, {'violator': 'Newberg Rock & Dirt', 'contract_id': '  ', 'citation_no': '8992214  ', 'case_no': '000435717', 'date_issues': '2/7/2017', 'final_order_date': '5/21/2017 ', 'section_of_act': '104(a)', 'date_terminated': '2/8/2017', 'citation': 'C', 's_and_s': 'N', 'standard': '56.14107(a)', 'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf', 'proposed_penalty': '116.00', 'citation_status': 'Closed', 'current_penalty': '11

### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [30]:
import pandas as pd

In [36]:
df = pd.DataFrame(data_list)
df.to_csv("3503598-violations.csv", index = False)

In [37]:
df1 = pd.read_csv("3503598-violations.csv")
df1.head()

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.0,440809,C,8992368,Closed,,116.0,5/2/2017,5/2/2017,7/23/2017,116.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
1,116.0,435717,C,8992214,Closed,,116.0,2/7/2017,2/8/2017,5/21/2017,116.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
2,100.0,398910,C,8872462,Closed,,100.0,10/21/2015,10/27/2015,1/13/2016,100.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
3,100.0,398910,C,8872461,Closed,,100.0,10/21/2015,10/27/2015,1/13/2016,100.0,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
4,100.0,383840,C,8790965,Closed,,100.0,4/15/2015,4/15/2015,7/17/2015,100.0,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt


## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

In [89]:
driver = webdriver.Chrome()
#Open link
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

#Search for mine
search_box = driver.find_element_by_name("MineId")
driver.execute_script("arguments[0].scrollIntoView(true)", search_box)
search_box.clear()
search_box.send_keys("3503598")

search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

#Search for violation
b_date = driver.find_element_by_name("BDate")
b_date.clear()
b_date.send_keys("1/1/1995")

violation_check = driver.find_elements_by_name("Submit")[3]
violation_check.click()

get_report = driver.find_elements_by_name("submit")[0]
get_report.submit()

#Scrape
results = driver.find_elements_by_css_selector("tr.drsviols")
data_list = []
for result in results:
    data = {}
    info = result.find_elements_by_tag_name("td")

    data['violator'] = info[0].text
    data['contract_id'] = info[1].text
    data['citation_no'] = info[2].text
    data['case_no'] = info[3].text
    data['date_issues'] = info[4].text
    data['final_order_date'] = info[5].text
    data['section_of_act'] = info[6].text
    data['date_terminated'] = info[7].text
    data['citation'] = info[8].text
    data['s_and_s'] = info[9].text
    data['standard'] = info[10].text
    data['standard_url'] = info[10].find_element_by_tag_name("a").get_attribute("href")
    data['proposed_penalty'] = info[11].text
    data['citation_status'] = info[12].text
    data['current_penalty'] = info[13].text
    data['amount_paid'] = info[14].text
    data_list.append(data)
    
#     print("my dictionary looks like", data)
#     print("-----------------")
print(data_list)

#Save df to csv
df = pd.DataFrame(data_list)
df.to_csv("3503598-violations.csv", index = False)

[{'violator': 'Newberg Rock & Dirt', 'contract_id': '  ', 'citation_no': '8992368  ', 'case_no': '000440809', 'date_issues': '5/2/2017', 'final_order_date': '7/23/2017 ', 'section_of_act': '104(a)', 'date_terminated': '5/2/2017', 'citation': 'C', 's_and_s': 'N', 'standard': '56.14107(a)', 'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf', 'proposed_penalty': '116.00', 'citation_status': 'Closed', 'current_penalty': '116.00 ', 'amount_paid': '116.00 '}, {'violator': 'Newberg Rock & Dirt', 'contract_id': '  ', 'citation_no': '8992214  ', 'case_no': '000435717', 'date_issues': '2/7/2017', 'final_order_date': '5/21/2017 ', 'section_of_act': '104(a)', 'date_terminated': '2/8/2017', 'citation': 'C', 's_and_s': 'N', 'standard': '56.14107(a)', 'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf', 'proposed_penalty': '116.00', 'citation_status': 'Closed', 'current_penalty': '11

In [125]:
df2 = pd.read_csv("3503598-violations.csv")
df2.tail()

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
44,100.0,121048,C,6342895,Closed,,100.0,5/23/2007,5/24/2007,8/17/2007,100.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2007-title30-...,Newberg Rock & Dirt
45,60.0,86543,C,6383438,Closed,,60.0,3/1/2006,3/2/2006,6/16/2006,60.0,N,104(a),56.11002,http://www.gpo.gov/fdsys/pkg/CFR-2006-title30-...,Newberg Rock & Dirt
46,60.0,86543,C,6383437,Closed,,60.0,3/1/2006,3/2/2006,6/16/2006,60.0,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2006-title30-...,Newberg Rock & Dirt
47,60.0,51247,C,6368407,Closed,,60.0,10/27/2004,1/21/2005,7/23/2005,60.0,N,104(a),62.120,http://www.gpo.gov/fdsys/pkg/CFR-2004-title30-...,Newberg Rock & Dirt
48,99.0,44260,C,6368406,Closed,,99.0,10/25/2004,10/27/2004,1/12/2005,99.0,Y,104(a),56.14131(a),http://www.gpo.gov/fdsys/pkg/CFR-2004-title30-...,Newberg Rock & Dirt


# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

In [76]:
df_subset = pd.read_csv("mines-subset.csv")
df_subset.dtypes
df_subset["id"] = df_subset["id"].apply(lambda x: '{0:0>7}'.format(x))

In [107]:
df_subset

,id
0,4104757
1,0801306
2,3609931


### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

In [176]:
data_list = []

def get_violation_info(row):
    #Open link
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")

    #Search for mine
    search_box = driver.find_element_by_name("MineId")
    driver.execute_script("arguments[0].scrollIntoView(true)", search_box)
    search_box.clear()
    search_box.send_keys(row["id"])

    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()

    #Search for violation
    b_date = driver.find_element_by_name("BDate")
    b_date.clear()
    b_date.send_keys("1/1/1995")

    violation_check = driver.find_elements_by_name("Submit")[3]
    violation_check.click()

    get_report = driver.find_elements_by_name("submit")[0]
    get_report.submit()
    
    #Scrape
    results = driver.find_elements_by_css_selector("tr.drsviols")
    for result in results:
        data = {}
        info = result.find_elements_by_tag_name("td")

        data['violator'] = info[0].text
        data['contract_id'] = info[1].text
        data['citation_no'] = info[2].text
        data['case_no'] = info[3].text
        data['date_issues'] = info[4].text
        data['final_order_date'] = info[5].text
        data['section_of_act'] = info[6].text
        data['date_terminated'] = info[7].text
        data['citation'] = info[8].text
        data['s_and_s'] = info[9].text
        data['standard'] = info[10].text
        try:
            data['standard_url'] = info[10].find_element_by_tag_name("a").get_attribute("href")
        except:
            data['standard_url'] = "no info"
        data['proposed_penalty'] = info[11].text
        try:
            data['citation_status'] = info[12].text
        except:
            data['citation_status'] = "no info"
        try:
            data['current_penalty'] = info[13].text
        except:
            data['current_penalty'] = "no info"
        try:    
            data['amount_paid'] = info[14].text
        except:
            data['amount_paid'] = "no info"
        
        data_list.append(data)
    #    print(1)
    #print("------------")
    return data_list


In [174]:
sub_list = df_subset.apply(get_violation_info, axis = 1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
------------
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
------------
1
1
1
1
1
1
1
1
------------


In [175]:
pd.DataFrame(sub_list[0])

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,100.00,000374480,C,8778046,Closed,,100.00,10/14/2014,10/14/2014,3/27/2015,100.00,N,104(a),56.14132(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
1,162.00,000374480,C,8778047,Closed,,162.00,10/14/2014,11/4/2014,3/27/2015,162.00,N,104(a),56.18010,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
2,243.00,000345454,C,8771783,Closed,,243.00,1/22/2014,1/22/2014,4/20/2014,243.00,Y,104(a),56.9300(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
3,100.00,000348280,C,8771781,Closed,,100.00,1/22/2014,1/22/2014,5/21/2014,100.00,N,104(a),56.14100(b),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
4,243.00,000345454,C,8771782,Closed,,243.00,1/22/2014,1/22/2014,4/20/2014,243.00,Y,104(a),56.9300(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
5,100.00,000345454,C,8771784,Closed,,100.00,1/22/2014,1/22/2014,4/20/2014,100.00,N,104(a),50.30(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
6,100.00,000316329,C,8688404,Closed,,100.00,1/29/2013,1/29/2013,4/19/2013,100.00,N,104(a),50.30(a),http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works
7,100.00,000316329,C,8681998,Closed,,100.00,1/28/2013,2/1/2013,4/19/2013,100.00,Y,104(a),56.9300(a),http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works
8,100.00,000316329,C,8681996,Closed,,100.00,1/28/2013,2/1/2013,4/19/2013,100.00,N,104(a),56.12028,http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works
9,162.00,000316329,C,8681995,Closed,,162.00,1/28/2013,2/1/2013,4/19/2013,162.00,N,104(a),56.14132(a),http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works


# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.

In [177]:
df_all_mines = pd.read_csv("mines.csv")
df_all_mines["id"] = df_all_mines["id"].apply(lambda x: '{0:0>7}'.format(x))
df_all_mines

,id
0,3503598
1,4801789
2,5001797
3,4608254
4,2103723
5,4104757
6,0801306
7,3901432
8,3609624
9,3609931


In [178]:
list_all_mines_with_info = df_all_mines.apply(get_violation_info, axis = 1)

In [183]:
df_all_mines_with_info = pd.DataFrame(list_all_mines_with_info[0])

In [186]:
df_all_mines_with_info.shape
df_all_mines_with_info.to_csv("all-mine-violations.csv", index = False)

In [ ]:
####
#IGNORE THIS PLZ
####


# df = []
# def get_violation_info(row):
#     df1 = []
#     #Open link
#     driver.get("https://arlweb.msha.gov/drs/drshome.htm")

#     #Search for mine
#     search_box = driver.find_element_by_name("MineId")
#     driver.execute_script("arguments[0].scrollIntoView(true)", search_box)
#     search_box.clear()
#     search_box.send_keys(row["id"])

#     search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
#     search_button.click()

#     #Search for violation
#     b_date = driver.find_element_by_name("BDate")
#     b_date.clear()
#     b_date.send_keys("1/1/1995")

#     violation_check = driver.find_elements_by_name("Submit")[3]
#     violation_check.click()

#     get_report = driver.find_elements_by_name("submit")[0]
#     get_report.submit()

#     #Scrape
#     results = driver.find_elements_by_css_selector("tr.drsviols")
#     for result in results:
#         info = result.find_elements_by_tag_name("td")

#         violator = info[0].text
#         contract_id = info[1].text
#         citation_no = info[2].text
#         case_no = info[3].text
#         date_issues = info[4].text
#         final_order_date = info[5].text
#         section_of_act = info[6].text
#         date_terminated = info[7].text
#         citation = info[8].text
#         s_and_s = info[9].text
#         standard = info[10].text
#         try:
#             standard_url = info[10].find_element_by_tag_name("a").get_attribute("href")
#         except:
#             standard_url = "no info"
#         proposed_penalty = info[11].text
#         try:
#             citation_status = info[12].text
#         except:
#             citation_status = "no info"
#         try:
#             current_penalty = info[13].text
#         except:
#             current_penalty = "no info"
#         try:    
#             amount_paid = info[14].text
#         except:
#             amount_paid = "no info"
#         df1 = pd.Series({
#             'violator' : violator,
#             'contract_id' : contract_id,
#             'citation_no' : citation_no,
#             'case_no' : case_no,
#             'date_issues' : date_issues,
#             'final_order_date' : final_order_date,
#             'section_of_act' : section_of_act,
#             'date_terminated' : date_terminated,
#             'citation' : citation,
#             's_and_s' : s_and_s,
#             'standard' : standard,
#             'standard_url' : standard_url,
#             'proposed_penalty' : proposed_penalty,
#             'citation_status' : citation_status,
#             'current_penalty' : current_penalty,
#             'amount_paid' : amount_paid
#         })
#         df = df.append(df1)
#     return df